In [9]:
from sklearn.metrics import classification_report
from tensorflow.keras import optimizers
from tensorflow.keras.callbacks import ModelCheckpoint, CSVLogger
from tensorflow.keras.preprocessing.image import ImageDataGenerator
#from tensorflow.keras.utils.layer_utils import print_summary
from tensorflow.keras.models import Model as KerasModel
from tensorflow.keras.layers import Input, Dense, Flatten, Conv2D, MaxPooling2D, BatchNormalization, Dropout, Layer, \
    Reshape, Concatenate, LeakyReLU
from tensorflow.keras.optimizers import Adam
import tensorflow as tf
from tensorflow.keras import backend as K
import matplotlib.pyplot as plt
import numpy as np
import os
import random
from pprint import pprint
#from Utility_functions import create_sequence, FreezeBatchNormalization
import pandas as pd
# from Utility_functions import FreezeBatchNormalization

from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint


In [10]:
def generate_generator_multiple(generator, dir1,  batch_size, img_height, img_width):
    genX1 = generator.flow_from_directory(dir1,
                                          target_size = (img_height, img_width),
                                          class_mode = 'binary',
                                          batch_size = batch_size,
                                          shuffle = True,
                                          seed = 32)
                                         

    i = 0
    while (True):
        if i < len(genX1.filepaths):
            X1i = genX1.next()
            yield X1i[0], X1i[1]

        i += 1


In [11]:
batch_size = 64
img_height = 256
img_width = 256
train_datagen = ImageDataGenerator(rescale = 1. / 255)
validation_datagen = ImageDataGenerator(rescale = 1. / 255)
test_datagen = ImageDataGenerator(rescale = 1. / 255)

train_generator = generate_generator_multiple(generator = train_datagen,
                                              dir1 = 'split_dataset/train',
                                             
                                              batch_size = batch_size,
                                              img_height = img_height,
                                              img_width = img_width)
validation_generator = generate_generator_multiple(validation_datagen,
                                                   dir1 = 'split_dataset/val',
                                                   
                                                   batch_size = batch_size,
                                                   img_height = img_height,
                                                   img_width = img_width)
test_generator = generate_generator_multiple(validation_datagen,
                                                   dir1 = 'split_dataset/test',
                                                   
                                                   batch_size = batch_size,
                                                   img_height = img_height,
                                                   img_width = img_width)


In [12]:
def get_pred(predictions):
    pred = []
    for p in predictions:
        if p < 0.50:
            pred.append(0)
        else:
            pred.append(1)
    return pred


IMGWIDTH = 256

In [13]:
class Classifier:
    def __init__(self):
        self.model = 0

    def predict(self, x):
        return self.model.predict(x)

    def fit(self, x, y):
        return self.model.train_on_batch(x, y)

    def get_accuracy(self, x, y):
        return self.model.test_on_batch(x, y)

    def load(self, path):
        self.model.load_weights(path)

In [14]:
class Meso4(Classifier):
    def __init__(self, learning_rate = 0.001):
        self.model = self.init_model()
        optimizer = Adam(lr = learning_rate)
        self.model.compile(optimizer = optimizer, loss = 'mean_squared_error', metrics = ['accuracy'])

    def init_model(self):
        x = Input(shape = (IMGWIDTH, IMGWIDTH, 3))

        x1 = Conv2D(8, (3, 3), padding = 'same', activation = 'relu')(x)
        x1 = BatchNormalization()(x1)
        x1 = MaxPooling2D(pool_size = (2, 2), padding = 'same')(x1)

        x2 = Conv2D(8, (5, 5), padding = 'same', activation = 'relu')(x1)
        x2 = BatchNormalization()(x2)
        x2 = MaxPooling2D(pool_size = (2, 2), padding = 'same')(x2)

        x3 = Conv2D(16, (5, 5), padding = 'same', activation = 'relu')(x2)
        x3 = BatchNormalization()(x3)
        x3 = MaxPooling2D(pool_size = (2, 2), padding = 'same')(x3)

        x4 = Conv2D(16, (5, 5), padding = 'same', activation = 'relu')(x3)
        x4 = BatchNormalization()(x4)
        x4 = MaxPooling2D(pool_size = (4, 4), padding = 'same')(x4)

        y = Flatten()(x4)
        y = Dropout(0.5)(y)
        y = Dense(16)(y)       
        y = LeakyReLU(alpha = 0.1)(y)
        y = Dropout(0.5)(y)
        y = Dense(1, activation = 'sigmoid')(y)

        return KerasModel(inputs = x, outputs = y)

In [15]:
model= Meso4()
# model.load("MesoNet1.h5")
model=model.model
is_training=True
top_k_layers=19
#model,df=FreezeBatchNormalization(is_training,top_k_layers,model)

checkpoint_filepath = '.\\tmp_checkpoint'
print('Creating Directory: ' + checkpoint_filepath)
os.makedirs(checkpoint_filepath, exist_ok=True)

optimizer = Adam(lr = 0.001)
model.compile(optimizer = optimizer, loss = 'mean_squared_error', metrics = ['accuracy'])

custom_callbacks = [
    EarlyStopping(
        monitor = 'val_loss',
        mode = 'min',
        patience = 5,
        verbose = 1
    ),
ModelCheckpoint(
    filepath = os.path.join(checkpoint_filepath, 'best_model.h5'),
    save_weights_only=True,
    monitor='val_acc',
    mode='max',
    save_best_only=True),
    
CSVLogger("./tmp_checkpoint/best_model.csv", append=True, separator=',')
#print_summary(model, line_length=115, positions=None, print_fn=None)
]

total_train = 4000
total_val =  4000
model.fit_generator(train_generator,
                    verbose = 1,
                    steps_per_epoch = int(total_train / batch_size),
                    epochs = 30,
                    validation_data = validation_generator,
                    validation_steps = int(total_val / batch_size),
                    callbacks=custom_callbacks)
model.save_weights("model/MesoNet3_50.h5")

print("model save succesfully")

Creating Directory: .\tmp_checkpoint


C:\Users\user\AppData\Local\Temp\ipykernel_13668\386316710.py:35: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  model.fit_generator(train_generator,


FileNotFoundError: [WinError 3] The system cannot find the path specified: 'split_dataset/train'

In [9]:
model.summary()

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 256, 256, 3)]     0         
                                                                 
 conv2d (Conv2D)             (None, 256, 256, 8)       224       
                                                                 
 batch_normalization (Batch  (None, 256, 256, 8)       32        
 Normalization)                                                  
                                                                 
 max_pooling2d (MaxPooling2  (None, 128, 128, 8)       0         
 D)                                                              
                                                                 
 conv2d_1 (Conv2D)           (None, 128, 128, 8)       1608      
                                                                 
 batch_normalization_1 (Bat  (None, 128, 128, 8)       32    

In [10]:
import pandas as pd

In [34]:
# load the saved model that is considered the best
from tensorflow.keras.models import load_model
filepath=".\\model"

# load the saved model that is considered the best
best_model = load_model(os.path.join(filepath, 'MesoNet3_50.h5'))

# Generate predictions
# test_generator.reset()

preds = best_model.predict(
    test_generator,
    verbose = 1
)

test_results = pd.DataFrame({
    "Filename": test_generator.filenames,
    "Prediction": preds.flatten()
})
print(test_results)

    235/Unknown - 70s 295ms/step

KeyboardInterrupt: 

In [32]:
model.save("model/MesoNet3_50.h5")


C:\ProgramData\Anaconda3\lib\site-packages\keras\src\engine\training.py:3000: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(
